### cnn_mnist_tflayers_tensorboard

In [ ]:
%matplotlib inline

import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

#########
# 신경망 모델 구성 - 모델의 뼈대 맞추기.
######


# placeholders
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='X')
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

# model
L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu)
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3, 3], activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 10, activation=None) 

# cost function and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)


# cost = cross entropy
cost_summ = tf.summary.scalar(name='cost', tensor=cost)


# training
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # Training the model
    log_dir = 'log/'
    board_dir = 'tf_board'
    train_writer = tf.summary.FileWriter(board_dir, sess.graph)
    summaries_train = tf.summary.merge_all()
    
    batch_size = 100
    total_batch = int(mnist.train.num_examples/batch_size) # n_batches
    
    
    for epoch_i in range(4): # range(num_epoch)
        total_cost = 0

        for batch_index in range(total_batch):
            features, labels = mnist.train.next_batch(batch_size)
            features = features.reshape(-1, 28, 28, 1)
            
            train_data = {X:features, Y:labels, is_training:True}
            
            if batch_index % 100 == 0: # Write to the log file every 100 iterations
                summary = cost_summ.eval(feed_dict=train_data)
                step = epoch_i * total_batch + batch_index
                print('Step %d: loss = %.2f'%(step, cost_val))
                train_writer.add_summary(summary, step)
            
            _, cost_val = sess.run([optimizer, cost], # optimizer : train_step
                                   feed_dict=train_data)
            
            total_cost += cost_val 
                
        print('Epoch: {}, \nLoss:{:.3f}'.format(epoch_i, 
                                                 (total_cost / total_batch)))
    train_writer.close()
    print('학습 완료!')
    

#########
# 결과 확인
######
    print('\n')
    prediction = tf.argmax(model, 1)
    target = tf.argmax(Y, 1)
    print('예측값:', sess.run(prediction, feed_dict={X: features}))
    print('실제값:', sess.run(target, feed_dict={Y: labels}))

    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: features, Y: labels}))

#### Launch tensorboard
chaeujeongui-MacBook-Pro:tf_board chaeujeong$ tensorboard --logdir=/Users/chaeujeong/tf_board